In [1]:
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.model import Trainer, Metadata, Interpreter

In [2]:
def train_nlu(data, configuration, model_dir):
    training_data = load_data(data)
    trainer = Trainer(config.load(configuration))
    trainer.train(training_data)
    model_directory = trainer.persist(model_dir,fixed_model_name='faq_chatbot_nlu')

In [3]:
train_nlu_model_switch = True

In [25]:
if (train_nlu_model_switch):
    train_nlu('./data/data.json','spacy_configuration.json','./models/faq_chatbot_nlu')

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished


In [26]:
def run_nlu(query):
    interpreter = Interpreter.load('./models/faq_chatbot_nlu/default/faq_chatbot_nlu')
    return interpreter.parse(query)

In [29]:
query = "frro location in cochin"

In [30]:
parsed_query = run_nlu(query)
parsed_query

C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\rasa_nlu\extractors\entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/faq_chatbot_nlu/default/faq_chatbot_nlu\entity_synonyms.json'
  "".format(entity_synonyms_file))


{'entities': [{'confidence': 0.9759997353306946,
   'end': 4,
   'entity': 'body',
   'extractor': 'ner_crf',
   'start': 0,
   'value': 'frro'},
  {'confidence': 0.6725072204381725,
   'end': 23,
   'entity': 'area',
   'extractor': 'ner_crf',
   'start': 17,
   'value': 'cochin'}],
 'intent': {'confidence': 0.92101726753129753, 'name': 'address'},
 'intent_ranking': [{'confidence': 0.92101726753129753, 'name': 'address'},
  {'confidence': 0.055591916592423948, 'name': 'greet'},
  {'confidence': 0.013783932451461712, 'name': 'goodbye'},
  {'confidence': 0.0096068834248169265, 'name': 'gratitude'}],
 'text': 'frro location in cochin'}